In [1]:
import pynq
from pynq_dpu import DpuOverlay

In [2]:
overlay = DpuOverlay("./dpu.bit")

In [3]:
overlay.load_model("./cae_2.5.xmodel")

In [4]:
import os
import time
import numpy as np
import threading
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
divider = '------------------------------------'

In [5]:
rails = pynq.get_rails()
rails

{'12V': Rail {name=12V, voltage=Sensor {name=12V_voltage, value=0.006V}, current=Sensor {name=12V_current, value=1.311A}, power=Sensor {name=12V_power, value=15.762W}},
 'in2': Rail {name=in2, voltage=Sensor {name=in2_voltage, value=12.02V}},
 'INT': Rail {name=INT, voltage=Sensor {name=INT_voltage, value=0.851V}, current=Sensor {name=INT_current, value=6.5A}, power=Sensor {name=INT_power, value=5.5W}},
 '1V8': Rail {name=1V8, voltage=Sensor {name=1V8_voltage, value=1.796V}, current=Sensor {name=1V8_current, value=0.234A}, power=Sensor {name=1V8_power, value=0.40625W}},
 '1V2': Rail {name=1V2, voltage=Sensor {name=1V2_voltage, value=1.199V}, current=Sensor {name=1V2_current, value=0.218A}, power=Sensor {name=1V2_power, value=0.25W}},
 'MGTA': Rail {name=MGTA, voltage=Sensor {name=MGTA_voltage, value=0.89V}, current=Sensor {name=MGTA_current, value=0.007A}, power=Sensor {name=MGTA_power, value=0.0W}},
 '3V3': Rail {name=3V3, voltage=Sensor {name=3V3_voltage, value=3.296V}, current=Senso

In [6]:
recorder1 = pynq.DataRecorder(rails['1V2'].power)
recorder2 = pynq.DataRecorder(rails['1V8'].power)
recorder3 = pynq.DataRecorder(rails['INT'].power)

In [7]:
import time
start = time.time()
recorder1.reset()
recorder2.reset()
recorder3.reset()
elapsed = 0
recorder1.record(0.5)
recorder2.record(0.5)
recorder3.record(0.5)
while elapsed < 30:
    elapsed = time.time() - start

recorder1.stop()
recorder2.stop()
recorder3.stop()


In [8]:
a1 = np.array(recorder1.frame)
a2 = np.array(recorder2.frame)
a3 = np.array(recorder3.frame)
print(a1.shape)
c1 = 0
c2 = 0
c3 = 0
for i in range(len(a1)):
    c1 = c1 + a1[i]
    c2 = c2 + a2[i]
    c3 = c3 + a3[i]

print(c1/len(a1))
print(c2/len(a2))
print(c3/len(a3))
total = (c1+c2+c3)/len(a1)
print('Total=',total)

(58, 2)
[0.         0.27693966]
[0.         0.39936441]
[0.         5.40677966]
Total= [0.         6.18318966]


In [9]:
    img = []
    runTotal = 1000
    global out_q
    out_q = [None] * runTotal
    print (divider)
    print('Pre-processing',runTotal,'images...')
    img = []
    test_X = np.zeros([1000,1,100,12,1])
    test_X = test_X.reshape([1000,1,100,12,1])#11
    for i in range(runTotal):
    #    path = os.path.join(image_dir,listimage[i])
        img.append(test_X[i])
    print(img[0])
    print(img[0].shape)

------------------------------------
Pre-processing 1000 images...
[[[[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  ...

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]
   ...
   [0.]
   [0.]
   [0.]]]]
(1, 100, 12, 1)


In [10]:
dpu = overlay.runner

In [11]:
def runDPU(id,start,dpu,img):
    '''get tensor'''
    inputTensors = dpu.get_input_tensors()
    outputTensors = dpu.get_output_tensors()
    input_ndim = tuple(inputTensors[0].dims)
    output_ndim = tuple(outputTensors[0].dims)
#     print('***Input tensor info***')
#     print(inputTensors)
#     print(input_ndim)
#     print('***Output tensor info***')
#     print(outputTensors)
#     print(output_ndim)

    batchSize = input_ndim[0]
    n_of_images = len(img)
    count = 0
    t3 = 0
    write_index = 0
    while count < n_of_images:
        if (count+batchSize<=n_of_images):
            runSize = batchSize
        else:
            runSize=n_of_images-count
        '''prepare batch input/output '''
        outputData = []
        inputData = []
       # t1 = time.time()
        inputData = [np.empty(input_ndim, dtype=np.int8, order="C")]
        outputData = [np.empty(output_ndim, dtype=np.int8, order="C")]
        #print(len(outputData))
        '''init input image to input buffer '''
        for j in range(runSize):
            imageRun = inputData[0]
            imageRun[j, ...] = img[(count + j) % n_of_images].reshape(input_ndim[1:])

        '''run with batch '''
        #print(inputData)
        #t1 = time.time()
        t1 = time.time()
        #print('here')
        job_id = dpu.execute_async(inputData,outputData)
#         #t1 = time.time()
        dpu.wait(job_id)
        t2 = time.time()
        #print(t2-t1)
        t3 = t3 + t2-t1
        #print('-----=',t3)
        #'''store output vectors '''
        #print(runSize)
        for j in range(runSize):
            out_q[write_index] = outputData
            print(outputData)
            write_index += 1
        count = count + runSize
        
    print (divider)
    print('Time per message =',t3/runTotal,'t3=',t3,'Total Messages=',len(img))
    print (divider)

In [12]:
    global a_file 
    threads = 1
    print('Starting',threads,'threads...')
    threadAll = []
    start=0
    recorder1.reset()
    recorder2.reset()
    recorder3.reset()
    recorder1.record(0.5)
    recorder2.record(0.5)
    recorder3.record(0.5)
    for i in range(threads):
        if (i==threads-1):
            end = len(img)
        else:
            end = start+(len(img)//threads)
        in_q = img[start:end]
        t1 = threading.Thread(target=runDPU, args=(i,start,dpu,in_q))
        threadAll.append(t1)
        start=end
    print('*** Time start ***')
    #t1 = np.zeros(50000)
    #t2 = np.zeros(50000)
    time1 = time.time()
    for x in threadAll:
        x.start()
    for x in threadAll:
        x.join()
    time2 = time.time()
    timetotal = time2 - time1

    fps = float(runTotal / timetotal)
    recorder1.stop()
    recorder2.stop()
    recorder3.stop()

#     #recorder.reset()
#     print (divider)
#     print("Throughput=%.2f fps, total frames = %.0f, time=%.4f seconds" %(fps, runTotal, timetotal))
#     print (divide

Starting 1 threads...
*** Time start ***
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
       

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

...

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [21]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]]]], dtype=int8)]
[array([[[[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        [[ 0],
         [ 0],
         [ 0],
         ...,
         [ 0],
         [ 0],
         [ 0]],

        ...,

  

In [13]:
a1 = np.array(recorder1.frame)
a2 = np.array(recorder2.frame)
a3 = np.array(recorder3.frame)
print(a1.shape)
c1 = 0
c2 = 0
c3 = 0
for i in range(len(a1)):
    c1 = c1 + a1[i]
    c2 = c2 + a2[i]
    c3 = c3 + a3[i]

print(c1/len(a1))
print(c2/len(a2))
print(c3/len(a3))
total = (c1+c2+c3)/len(a1)
print(total)

(5, 2)
[0.      0.36875]
[0.         0.33854167]
[0.   4.75]
[0.    6.475]


In [14]:
del overlay
del dpu